In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime

In [2]:
'''
offline_data

shape = (1754884, 7)
真正用户数量 = (539438, 6)

Coupon_id = null 表示无优惠券消费，此时 Discount_rate 和 Date_received 字段无意义
Discount_rate 有 2 种情况：[0, 1] - 折扣率， x:y - 满x减y 
Distance 最近商铺距离 [0, 10]
样本分为 3 类： 负样本                          无影响样本                       正样本
                Date=null & Coupon_id!=null     Date!=null & Coupon_id==null     Date!=null & Coupon_id != null
                
负样本 = (977900, 7)  (0,1)， 代表领取优惠券行为
    有过领取行为的人数 = (497820, 7)， 占比 92%
正样本 = (497820, 7)    (1, 1)，表示用优惠券消费, 会有同一个用户多次使用优惠券消费
    有过使用消费券行为的人数  = (46395, 7)，占比 9%
无影响样本 = (701602, 7) (1, 0)， 代表正常消费
    有过正常消费的人数 (207619, 7)， 占比 38 %
所有都是有效数据，没有出现 (0, 0) 的情况

so true_负样本 = 所有人 - (有领取过优惠券的人 & 有使用过优惠券行为的人) or (有领取过优惠券的人 & 没有使用优惠券行为的人) 过严厉
   true_正样本 = (有领取过优惠券的人 & 有使用过优惠券行为的人)  =>  size =(370900, 7) 过宽松

pos_rate = 0.09
neg_rate = 0.92
norm_rate = 0.41

'''

offline_data = pd.read_csv("data/offline.csv")
offline_data.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


In [3]:
'''
oneline_data

shape = (11429826, 7)
真正用户数量 = (762858, 6)

Coupon_id = null 表示无优惠券消费，此时 Discount_rate 和 Date_received 字段无意义, 'fixed' 代表低价促销
Discount_rate 有 2 种情况：[0, 1] - 折扣率， x:y - 满x减y 
Distance 最近商铺距离 [0, 10]
样本分为 3 类： 负样本                          无影响样本                       正样本
                Date=null & Coupon_id!=null     Date!=null & Coupon_id==null     Date!=null & Coupon_id != null
                

负样本 = (655898, 7)  (0,1)， 代表领取优惠券行为
    有领取过优惠券的人数 = (497820, 7)， 占比 28%
    数据与 Action = 2 的数据是一致的
正样本 = (216459, 7)    (1, 1)，表示用优惠券消费
    有使用过优惠券行为的人数 = (95655, 7)， 占比 13%
无影响样本 = (10557469, 7) (1, 0)， 代表正常消费
    有过正常消费的人数 = (726146, 7), 占比 95%
    
so true_负样本 = 所有人 - (有领取过优惠券的人 & 有使用过优惠券行为的人)
   true_正样本 = (有领取过优惠券的人 & 有使用过优惠券行为的人)
   
   
所有都是有效数据，没有出现 (0, 0) 的情况
低价促销数量 (131546, 7)

pos_rate = 0.01
neg_rate = 0.06
norm_rate = 0.93

'''

online_data = pd.read_csv("data/online.csv")
online_data.head()

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,13740231,18907,2,100017492,500:50,20160513,null
1,13740231,34805,1,null,null,null,20160321
2,14336199,18907,0,null,null,null,20160618
3,14336199,18907,0,null,null,null,20160618
4,14336199,18907,0,null,null,null,20160618


In [3]:
'''
载入测试数据
'''
test_data = pd.read_csv('data/test_revised.csv')

In [43]:
# 得到线上有过领取优惠券行为的用户id
offline_recived_user = offline_data.loc[(offline_data['Coupon_id'] != 'null') & (offline_data['Date'] == 'null')]['User_id'].tolist()
offline_recived_user_id = list(set(offline_recived_user))

# 得到有过领取优惠券行为的用户的所有记录（id 有重复）
offline_recived_user_data = offline_data.loc[offline_data['User_id'].isin(offline_recived_user_id)]
online_revived_user_data = online_data.loc[online_data['Action'] == 2]

print("offline_recived_user_id size = ", len(offline_recived_user_id))

offline_recived_user_id size =  497820


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


In [46]:
offline_used_user_id_list = offline_data.loc[(offline_data['Coupon_id'] != 'null') & (offline_data['Date'] != 'null')]['User_id'].tolist()
offline_used_user_id = list(set(offline_used_user_id_list))
print("offline_used_user_id size = ", len(offline_used_user_id))

# 线下没有使用过优惠券的人 = 全部人 - 使用过的人 = （领取过的 + 正常消费的）
total_user_id_set = set(offline_data['User_id'].tolist())
offline_not_used_user_id_list = total_user_id_set - set(offline_used_user_id)
offline_not_used_user_id = list(offline_not_used_user_id_list)

received_and_used_data = offline_recived_user_data.loc[offline_recived_user_data['User_id'].isin(offline_used_user_id)]
received_and_used_user_id = list(set(received_and_used_data['User_id'].tolist()))
print("received_and_used_user_id size = ", len(received_and_used_user_id))
received_but_used_data = offline_recived_user_data.loc[offline_recived_user_data['User_id'].isin(offline_not_used_user_id)]
received_but_used_user_id = list(set(received_but_used_data['User_id'].tolist()))
print("received_but_used_user_id size = ", len(received_but_used_user_id))

offline_used_user_id size =  46395
received_and_used_user_id size =  33517
received_but_used_user_id size =  464303


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null
5,1439408,2632,null,null,0,null,20160516
6,1439408,2632,8591,20:1,0,20160516,20160613


In [4]:
'''
重新提取离线数据
'''

offline_received_but_used = offline_data.loc[(offline_data['Date'] == 'null') & (offline_data['Coupon_id'] != 'null')]
offline_received_and_used = offline_data.loc[(offline_data['Date'] != 'null') & (offline_data['Coupon_id'] != 'null')]
print("offline_received_but_used size = ", offline_received_but_used.shape)
print("offline_received_and_used size = ", offline_received_and_used.shape)

offline_received_and_used['Date_received'] = offline_received_and_used['Date_received'].apply(lambda row: datetime.datetime.strptime(row, '%Y%m%d'))
offline_received_and_used['Date'] = offline_received_and_used['Date'].apply(lambda row: datetime.datetime.strptime(row, '%Y%m%d'))
offline_received_and_used['Date_delta'] = (offline_received_and_used['Date'] - offline_received_and_used['Date_received'])
offline_received_and_used['Date_delta'] = offline_received_and_used['Date_delta'].apply(lambda row: row.days)

offline_positive_data = offline_received_and_used.loc[offline_received_and_used['Date_delta'] <= 15]
offline_negative_data = offline_received_and_used.loc[offline_received_and_used['Date_delta'] > 15]

print("offline_positive_data size = ", offline_positive_data.shape)
print("offline_negative_data", offline_negative_data.shape)

offline_received_but_used size =  (977900, 7)
offline_received_and_used size =  (75382, 7)


D:\DL\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
D:\DL\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
D:\DL\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

offline_positive_data size =  (64395, 8)
offline_negative_data (10987, 8)


D:\DL\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [41]:
'''
重新提取在线数据
'''
online_received_but_used = online_data.loc[(online_data['Date'] == 'null') & (online_data['Coupon_id'] != 'null')]
print("offline_received_but_used size = ", online_received_but_used.shape)
online_received_and_used = online_data.loc[(online_data['Date'] != 'null') & (online_data['Coupon_id'] != 'null')]
print("online_received_and_used size = ", online_received_and_used.shape)

online_received_and_used['Date_received'] = online_received_and_used['Date_received'].apply(lambda row: datetime.datetime.strptime(row, '%Y%m%d'))
online_received_and_used['Date'] = online_received_and_used['Date'].apply(lambda row: datetime.datetime.strptime(row, '%Y%m%d'))
online_received_and_used['Date_delta'] = (online_received_and_used['Date'] - online_received_and_used['Date_received'])
online_received_and_used['Date_delta'] = online_received_and_used['Date_delta'].apply(lambda row: row.days)

online_positive_data = online_received_and_used.loc[online_received_and_used['Date_delta'] <= 15]
online_negative_data = online_received_and_used.loc[online_received_and_used['Date_delta'] > 15]

print("online_positive_data size = ", online_positive_data.shape)
print("online_negative_data", online_negative_data.shape)


offline_received_but_used size =  (655898, 7)
online_received_and_used size =  (216459, 7)


D:\DL\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
D:\DL\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
D:\DL\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

online_positive_data size =  (212976, 8)
online_negative_data (3483, 8)


D:\DL\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [5]:
'''
    正样本 = 领取了优惠券并且在15天内用了的 = 40075
    负样本 = 领取了优惠券但没在15天内用了的 = 9514
    其中有重复样本 （既有正样本行为，也有负样本行为）
'''

offline_positive_user_id = list(set(offline_positive_data['User_id'].tolist()))
offline_negative_user_id = list(set(offline_negative_data['User_id'].tolist()))
print(len(offline_positive_user_id))
print(len(offline_negative_user_id))

40075
9514


In [6]:
# 将领取日期转换为与记录起始日期的间隔
# 可能某一段时间大家都有可能用优惠券
start_date = datetime.datetime.strptime('20160101', '%Y%m%d')
offline_positive_data.loc[:, 'Date_received'] = offline_positive_data['Date_received'].apply(lambda row: (row - start_date).days)
offline_positive_data.head()

D:\DL\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Date_delta
33,1113008,1361,11166,20:1,0,135,2016-05-21,6
38,2881376,8390,7531,20:5,0,80,2016-03-29,8
69,114747,6901,2366,30:5,0,143,2016-06-05,13
76,114747,5341,111,30:5,0,37,2016-02-18,11
77,114747,5341,7751,50:10,0,26,2016-01-28,1


In [7]:
# 将 Date_received 转换为与记录起始日期的间隔
# 可能某一段时间大家都有可能用优惠券
start_date = datetime.datetime.strptime('20160101', '%Y%m%d')
offline_negative_data.loc[:, 'Date_received'] = offline_negative_data['Date_received'].apply(lambda row: (row - start_date).days)
offline_negative_data.head()

D:\DL\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,Date_delta
6,1439408,2632,8591,20:1,0,136,2016-06-13,28
75,114747,5341,111,30:5,0,26,2016-02-21,25
100,114747,3381,9776,10:5,0,28,2016-02-21,23
368,1910192,8244,5756,30:5,10,148,2016-06-26,29
426,222550,3381,9776,10:5,0,28,2016-02-14,16


In [8]:
# 删除 positive 和 negative 的 Date 和 Date_delta 两列
temp_positive_data = offline_positive_data.drop(['Date', 'Date_delta'], axis=1)
temp_negative_data = offline_negative_data.drop(['Date', 'Date_delta'], axis=1)

# 加上对应的数据标签 （0-负样本， 1-正样本）
pos_label = np.ones(temp_positive_data.shape[0], dtype=np.int8)
neg_label = np.zeros(temp_negative_data.shape[0], dtype=np.int8)
temp_positive_data['label'] = pos_label
temp_negative_data['label'] = neg_label

In [9]:
'''
处理 Distance
'''

# 把 Distance 中的 null 替换为 11
temp_negative_data.loc[temp_negative_data['Distance'] == 'null', 'Distance'] = 11
temp_positive_data.loc[temp_positive_data['Distance'] == 'null', 'Distance'] = 11

In [10]:
'''
处理 Discout_rate
'''

# 处理 positive_data 的 Discout_rate
# 把 Discount_rate 切分
a = temp_positive_data['Discount_rate'].str.split(':', expand=True)
a.columns = ['achieve', 'reduce']
a['achieve'] = pd.to_numeric(a['achieve'])

# 填充 reduce == null，即源数据是折扣比例
a['reduce'].fillna(0, inplace=True)
a['reduce'] = pd.to_numeric(a['reduce'])

# 生成 满减比例
a['Coupon_rate'] = a['reduce'] / a['achieve']
a['Discount_rate'] = a['achieve']
a.loc[a['Coupon_rate'] != 0, 'Discount_rate'] = 0

a.drop(['reduce'], axis=1, inplace=True)
a.loc[a['Discount_rate'] != 0, 'achieve'] = 0

# 处理 negative_data 的 Discount_rate

# 把 Discount_rate 切分
b = temp_negative_data['Discount_rate'].str.split(':', expand=True)
b.columns = ['achieve', 'reduce']
b['achieve'] = pd.to_numeric(b['achieve'])

# 填充 reduce == null，即源数据是折扣比例
b['reduce'].fillna(0, inplace=True)
b['reduce'] = pd.to_numeric(b['reduce'])

# 生成 满减比例
b['Coupon_rate'] = b['reduce'] / b['achieve']
b['Discount_rate'] = b['achieve']
b.loc[b['Coupon_rate'] != 0, 'Discount_rate'] = 0

b.drop(['reduce'], axis=1, inplace=True)
b.loc[b['Discount_rate'] != 0, 'achieve'] = 0

# 合并两个DataFrame
temp_positive_data.drop(['Discount_rate'], axis=1, inplace=True)
temp_negative_data.drop(['Discount_rate'], axis=1, inplace=True)

temp_positive_data = pd.concat([temp_positive_data, a], axis=1)
temp_negative_data = pd.concat([temp_negative_data, b], axis=1)

temp_positive_data = temp_positive_data[['User_id', 'Merchant_id', 'Coupon_id', 'Distance', 'Date_received', 'achieve', 'Coupon_rate', 'Discount_rate', 'label']]
temp_negative_data = temp_negative_data[['User_id', 'Merchant_id', 'Coupon_id', 'Distance', 'Date_received', 'achieve', 'Coupon_rate', 'Discount_rate', 'label']]

In [11]:
'''
处理 Merchant ID 
预测数据中的 Merchant 都没出现在训练集中，因此删去 Merchant_id
'''
train_merchant_id = set(offline_data['Coupon_id'].tolist())
test_merchant_id = set(test_data['Coupon_id'].tolist())
res = len((test_merchant_id & train_merchant_id))
test_size = len(test_merchant_id)
print(res)
print(test_size)

temp_positive_data.drop(['Merchant_id'], axis=1, inplace=True)
temp_negative_data.drop(['Merchant_id'], axis=1, inplace=True)

0
2050


In [12]:
'''
处理 Coupon_id
预测数据中的 Coupon_id 都没出现在训练集中，因此删去 Coupon_id
'''
test_coupon_id = set(test_data['Coupon_id'].tolist())
train_coupon_id = set(offline_data['Coupon_id'].tolist())
print(len((test_coupon_id & train_coupon_id)))
print(len(test_coupon_id))

temp_positive_data.drop(['Coupon_id'], axis=1, inplace=True)
temp_negative_data.drop(['Coupon_id'], axis=1, inplace=True)

0
2050


In [13]:
'''
处理 User_id 和 Date_received ，暂时用不到
'''
temp_positive_data.drop(['Date_received', 'User_id'], axis=1, inplace=True)
temp_negative_data.drop(['Date_received', 'User_id'], axis=1, inplace=True)

In [14]:
subsample = temp_positive_data.sample(n=20000, random_state=0)
subsample.head()

,Distance,achieve,Coupon_rate,Discount_rate,label
1006630,0,30.0,0.166667,0.0,1
1705540,1,30.0,0.166667,0.0,1
614046,1,10.0,0.500000,0.0,1
1189293,0,50.0,0.100000,0.0,1
656628,0,10.0,0.100000,0.0,1


In [15]:
'''
初步合并数据进行 BaseLine 的测试
'''
train = pd.concat([temp_positive_data, temp_negative_data], axis=0)
# train = pd.concat([subsample, temp_negative_data], axis=0)
train.shape

(75382, 5)

In [16]:
'''
划分训练集，验证集
'''
from sklearn.model_selection import train_test_split
X = train.iloc[:, 0:4]
y = train['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=0)

In [18]:
'''
使用 Logistics Regression
'''
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=0)
lr = LogisticRegression(penalty='l1', C=0.1)
scores_cvs = cross_validate(lr, X, y, cv=5, scoring='roc_auc', return_train_score = True)
sorted(scores_cvs.keys())
print(scores_cvs['test_score'])

[0.66638143 0.65942004 0.63867222 0.65294879 0.65434985]


In [18]:
'''
使用 GBDT
'''
from  sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
clf = GradientBoostingClassifier(learning_rate=0.01, n_estimators=100, max_depth=10)
cv = KFold(n_splits=5, shuffle=True, random_state=0)
scores_cvs = cross_validate(clf, X, y, cv=5, scoring='roc_auc', return_train_score = True)
sorted(scores_cvs.keys())
print(scores_cvs['test_score'])

[0.68000033 0.68528871 0.66741738 0.68447976 0.68468022]


In [20]:
'''
使用SVM
'''
from sklearn.svm import SVC

svc = SVC(C=0.01)
cv = KFold(n_splits=5, shuffle=True, random_state=0)
scores_cvs = cross_validate(svc, X, y, cv=5, scoring='roc_auc', return_train_score = True)
sorted(scores_cvs.keys())
print(scores_cvs['test_score'])

[0.61071491 0.6304975  0.62716483 0.64147576 0.61990026]


In [29]:
from sklearn.metrics import roc_auc_score
clf.fit(X_train, y_train)
pred = clf.predict_proba(X_test)

In [31]:
res = pd.DataFrame(pred, columns=['neg', 'pos'])
# res['pos'] = res['pos'].apply(lambda row: 1 if  row>0.857 else 0)
pred = res['pos'].tolist()
roc_auc_score(y_test, pred)
# roc_auc_score(y_test, pred)

0.6887754838234831

In [33]:
test_data.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1,20160712
1,6949378,1300,3429,30:5,null,20160706
2,2166529,7113,6928,200:20,5,20160727
3,2166529,7113,1808,100:10,5,20160727
4,6172162,7605,6500,30:1,2,20160708


In [19]:
'''
处理预测数据的 Discount_rate
'''
a = test_data['Discount_rate'].str.split(':', expand=True)
a.columns = ['achieve', 'reduce']
a['achieve'] = pd.to_numeric(a['achieve'])

# 填充 reduce == null，即源数据是折扣比例
a['reduce'].fillna(0, inplace=True)
a['reduce'] = pd.to_numeric(a['reduce'])

# 生成 满减比例
a['Coupon_rate'] = a['reduce'] / a['achieve']
a['Discount_rate'] = a['achieve']
a.loc[a['Coupon_rate'] != 0, 'Discount_rate'] = 0

a.drop(['reduce'], axis=1, inplace=True)
a.loc[a['Discount_rate'] != 0, 'achieve'] = 0

# 合并两个DataFrame
test_data.drop(['Discount_rate'], axis=1, inplace=True)
test_data = pd.concat([test_data, a], axis=1)

test_data.head()

,User_id,Merchant_id,Coupon_id,Distance,Date_received,achieve,Coupon_rate,Discount_rate
0,4129537,450,9983,1,20160712,30.0,0.166667,0.0
1,6949378,1300,3429,null,20160706,30.0,0.166667,0.0
2,2166529,7113,6928,5,20160727,200.0,0.100000,0.0
3,2166529,7113,1808,5,20160727,100.0,0.100000,0.0
4,6172162,7605,6500,2,20160708,30.0,0.033333,0.0


In [20]:
'''
处理预测数据的Distance
'''
# 把 Distance 中的 null 替换为 11
test_data.loc[test_data['Distance'] == 'null', 'Distance'] = 11
test_data.head()

,User_id,Merchant_id,Coupon_id,Distance,Date_received,achieve,Coupon_rate,Discount_rate
0,4129537,450,9983,1,20160712,30.0,0.166667,0.0
1,6949378,1300,3429,11,20160706,30.0,0.166667,0.0
2,2166529,7113,6928,5,20160727,200.0,0.100000,0.0
3,2166529,7113,1808,5,20160727,100.0,0.100000,0.0
4,6172162,7605,6500,2,20160708,30.0,0.033333,0.0


In [21]:
'''
去除暂时无关的 User_id, Merchant_id, Coupon_id, Date_received
'''
test_data.drop(['User_id', 'Merchant_id', 'Coupon_id', 'Date_received'], axis=1, inplace=True)
test_data.head()

,Distance,achieve,Coupon_rate,Discount_rate
0,1,30.0,0.166667,0.0
1,11,30.0,0.166667,0.0
2,5,200.0,0.100000,0.0
3,5,100.0,0.100000,0.0
4,2,30.0,0.033333,0.0


In [22]:
'''
使用训练好的 GBDT 进行数据的预测
'''
clf.fit(X_train, y_train)
pred = clf.predict_proba(X_test)

In [23]:
'''
将预测结果和数据信息结合，生成最终的结果DataFrame
'''
pred_data = pd.DataFrame(pred, columns=['Probability', 'neg'])
pred_data.drop(['neg'], axis=1, inplace=True)
test = pd.read_csv('data/test_revised.csv')
temp = test.loc[:, ['User_id', 'Coupon_id', 'Date_received']]
res = pd.concat([pred_data, temp], axis=1)
res = res[['User_id', 'Coupon_id', 'Date_received', 'Probability']]
res.head()

,User_id,Coupon_id,Date_received,Probability
0,4129537,9983,20160712,0.139267
1,6949378,3429,20160706,0.054031
2,2166529,6928,20160727,0.219894
3,2166529,1808,20160727,0.139267
4,6172162,6500,20160708,0.139267


In [24]:
'''
生成最后的提交文件
'''
res.to_csv('result.csv', index=False)

In [59]:
test['Date_received'].value_counts()

20160703    5491
20160704    5121
20160723    4853
20160725    4746
20160726    4701
20160705    4693
20160706    4574
20160711    4521
20160724    4507
20160712    4349
20160727    4156
20160710    4100
20160713    4082
20160728    3906
20160702    3831
20160701    3808
20160714    3655
20160722    3613
20160729    3370
20160715    3290
20160718    3284
20160717    3213
20160719    3128
20160721    3044
20160716    3002
20160720    2984
20160730    2258
20160707    2234
20160731    2094
20160709    1616
20160708    1416
Name: Date_received, dtype: int64